In [1]:
import pandas as pd
import jieba
from sklearn.metrics import jaccard_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import re
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import os 

In [2]:
# # 初始化中文模型和分词器
# model_path = "/Users/liuhaoyu/Documents/bert4ner-base-chinese"

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForTokenClassification.from_pretrained(model_path)

In [3]:
#读取文件
data_original = pd.read_excel('../input folder/公牛原帖_作者id合并后.xlsx')
data_original

,笔记id,作者昵称,作者id,作者链接,作品链接,作品标题,发布时间,图片数量,图片列表,关联话题名称,...,小红书号,用户ip,简介,location,关注,粉丝,获赞与收藏,info,profession,college
0,65ab78e7000000000a030984 65ddc72f000000000b00edeb,鱼的宇宙,52732665b4c4d6556e9b39f1,https://www.xiaohongshu.com/user/profile/52732...,https://www.xiaohongshu.com/explore/65ab78e700...,2岁宝宝的阅读角 床头柜区域改造 边住边改之：电视柜改绘本架,2024-01-20 15:40:23,7,http://sns-webpic-qc.xhscdn.com/202403252132/c...,#儿童书架##家居改造##房间改造##亲子共读##读书角布置##读书角##绘本架##惬意时光...,...,104571957,北京,工作之外，才是生活。,北京朝阳区,312,43,38,女 女,视觉设计师,NaN
1,6022733c0000000021037a82 6587fdce0000000009018...,甜橙小布丁🍮,52e0fc98b4c4d626cd1fa18e,https://www.xiaohongshu.com/user/profile/52e0f...,https://www.xiaohongshu.com/explore/6022733c00...,环球影城｜哈利波特｜魔杖触发事件 就在武汉！人少！好看！免费！的国风美术馆 左滑查看成片！成...,2021-02-09 19:34:20,148,http://sns-webpic-qc.xhscdn.com/202403252239/c...,#武汉吃喝玩乐##武汉旅游攻略##武汉拍照##武汉打卡##周末去哪儿##宝藏店铺##假日c...,...,958412046,湖北,💕💕💕,NaN,396,1629,550980,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,摄影博主,NaN
2,658d1a28000000001d0303e4 6592716d000000001101d...,大耳朵猫酱,52e5fca0b4c4d61c32e520d2,https://www.xiaohongshu.com/user/profile/52e5f...,https://www.xiaohongshu.com/explore/658d1a2800...,"OAKMOO""忍冬灵药""～小个子天菜 新年第一天的快乐来自Moi Par暖 生椰拿铁get ...",2023-12-28 14:48:08,103,http://sns-webpic-qc.xhscdn.com/202403252335/5...,#王者毅也的笔记#,...,100870633,广东,都是自用分享，别总蔽我，好吗👌,神秘,543,141,203480,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,NaN,NaN
3,65bde01900000000010310d3 65be15640000000011000...,🍬美丽心情永恒的阳光,52f38137b4c4d66b2eaeb2b6,https://www.xiaohongshu.com/user/profile/52f38...,https://www.xiaohongshu.com/explore/65bde01900...,祖玛珑香水推荐 周大福这款怎么不火？！ 今天武汉大雪🌨️ 过春节能干嘛之写字✍🏻 过春节在家...,2024-02-03 14:41:29,59,http://sns-webpic-qc.xhscdn.com/202403252147/f...,#祖玛珑##苦橙##香水推荐##英国梨与小苍兰# #周大福##周大福传承##周大福黄金# #...,...,951028702,湖北,ENTP丨斜杠青年丨体制内\n分享型人格丨喜欢一切美好事物\n…………\n“像一个新的大人，...,NaN,1115,128,31260,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,文化博主,NaN
4,63614f340000000007037280 6364c1de0000000008013...,满星欢喜,52ff7f7eb4c4d65a7e9329e4,https://www.xiaohongshu.com/user/profile/52ff7...,https://www.xiaohongshu.com/explore/63614f3400...,40块搞定玄关摆件 挂画太俗？玄关摆件第二弹～200元的快乐 为新家配置了三口锅，满足日常所...,2022-11-02 00:54:12,105,http://sns-webpic-qc.xhscdn.com/202403252207/b...,#玄关摆件##花器##玄关画##玄关##入户玄关##玄关设计##我的装修记录# #玄关过道走...,...,180263222,福建,装修小白一只\n家装完成啦 已入住一年多\n偶尔分享一些生活好物 \n随喜随性 祝大家生活愉快,NaN,146,167,189380,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,家居博主,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22034,65fe9d60000000000d00f160 65fedc52000000000d00f...,毕,65fd3a6d000000000600c11f,https://www.xiaohongshu.com/user/profile/65fd3...,https://www.xiaohongshu.com/explore/65fe9d6000...,劳动人就要戴劳力士，我也一劳永逸了！ 个人急出，年前朋友送的不太喜欢了！忍痛出 急出！！含泪...,2024-03-23 17:14:08,15,http://sns-webpic-qc.xhscdn.com/202403252325/9...,#劳力士##劳力士日志##劳力士rolex# #日志##手表# #女表##日志# #劳力士#...,...,42762125897,福建,还没有简介,NaN,0,1,126,,NaN,NaN
22035,65ffe1c00000000012021877 6600da76000000000d00dee7,七里香,65fd56d5000000000600c62b,https://www.xiaohongshu.com/user/profile/65fd5...,https://www.xiaohongshu.com/explore/65ffe1c000...,"看了男友手机 我刷他卡买了块冰莓粉 却.... 个人急售！！忍痛亏出,暂挂3天",2024-03-24 16:18:08,5,http://sns-webpic-qc.xhscdn.com/202403252327/4...,#劳力士##rolex# #日志##女表#,...,26315541830,福建,还没有简介,NaN,0,1,22,,NaN,NaN
22036,65fdb3ed000000001203ebaf 65fdbcfe000000000d00f...,煎饼果子,65fdb31b000000000600c4ac,https://www.xiaohongshu.com/user/profile/65fdb...,https://www.xiaohongshu.com/explore/65fdb3ed00...,现代艺术,2024-03-23 00:38:05,17,http://sns-webpic-qc.xhscdn.com/202403252135/f...,,...,42767838188,广东,还没有简介,NaN,3,1,78,,NaN,NaN
22037,66013bf60000000012023c8d 66013d80000000000d00c...,山橘,65fe598a000000000b030ea1,https://www.xiaohongshu.com/user/profile/65fe5...,https://www.xiaohongshu.com/explore/66013bf600...,记录一下人流过程 做完人流手术，分享一下过程 分手后，发现怀孕，就做了人流手术 做人流手术，...,2024-03-25 16:55:18,5,http://sns-webpic-qc.xhscdn.com/202403252358/d...,,...,11889112600,安徽,沉默是金,NaN,0,0,5,,NaN,NaN


In [4]:
# #清洗重复数据
# data = data_original.drop_duplicates()
# #使用fuzzywuzzy来根据某列中重复的数据来筛选
# def find_duplicates(series, threshold=90):
#     duplicates = []
#     for i, item in enumerate(series):
#         if process.extractOne(item, series[i+1:])[1] > threshold:
#             duplicates.append(True)
#         else:
#             duplicates.append(False)
#     return duplicates
# data['帖子标题重复'] = find_duplicates(data['帖子标题'])
# data_filtered = data[~data['帖子标题重复']]
# data_filtered

In [5]:
column_headers = data_original.columns.tolist()
column_headers

['笔记id',
 '作者昵称',
 '作者id',
 '作者链接',
 '作品链接',
 '作品标题',
 '发布时间',
 '图片数量',
 '图片列表',
 '关联话题名称',
 '作品正文内容',
 '小红书号',
 '用户ip',
 '简介',
 'location',
 '关注',
 '粉丝',
 '获赞与收藏',
 'info',
 'profession',
 'college']

In [6]:
#清洗数据，删除重复值和空值
# data_original['帖子标题']
# data = data_original['帖子标题'].dropna()
# data = data_original['帖子标题'].drop_duplicates()

clean_data = data_original.dropna(subset=['作品标题', '作品正文内容'])

clean_data

,笔记id,作者昵称,作者id,作者链接,作品链接,作品标题,发布时间,图片数量,图片列表,关联话题名称,...,小红书号,用户ip,简介,location,关注,粉丝,获赞与收藏,info,profession,college
0,65ab78e7000000000a030984 65ddc72f000000000b00edeb,鱼的宇宙,52732665b4c4d6556e9b39f1,https://www.xiaohongshu.com/user/profile/52732...,https://www.xiaohongshu.com/explore/65ab78e700...,2岁宝宝的阅读角 床头柜区域改造 边住边改之：电视柜改绘本架,2024-01-20 15:40:23,7,http://sns-webpic-qc.xhscdn.com/202403252132/c...,#儿童书架##家居改造##房间改造##亲子共读##读书角布置##读书角##绘本架##惬意时光...,...,104571957,北京,工作之外，才是生活。,北京朝阳区,312,43,38,女 女,视觉设计师,NaN
1,6022733c0000000021037a82 6587fdce0000000009018...,甜橙小布丁🍮,52e0fc98b4c4d626cd1fa18e,https://www.xiaohongshu.com/user/profile/52e0f...,https://www.xiaohongshu.com/explore/6022733c00...,环球影城｜哈利波特｜魔杖触发事件 就在武汉！人少！好看！免费！的国风美术馆 左滑查看成片！成...,2021-02-09 19:34:20,148,http://sns-webpic-qc.xhscdn.com/202403252239/c...,#武汉吃喝玩乐##武汉旅游攻略##武汉拍照##武汉打卡##周末去哪儿##宝藏店铺##假日c...,...,958412046,湖北,💕💕💕,NaN,396,1629,550980,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,摄影博主,NaN
2,658d1a28000000001d0303e4 6592716d000000001101d...,大耳朵猫酱,52e5fca0b4c4d61c32e520d2,https://www.xiaohongshu.com/user/profile/52e5f...,https://www.xiaohongshu.com/explore/658d1a2800...,"OAKMOO""忍冬灵药""～小个子天菜 新年第一天的快乐来自Moi Par暖 生椰拿铁get ...",2023-12-28 14:48:08,103,http://sns-webpic-qc.xhscdn.com/202403252335/5...,#王者毅也的笔记#,...,100870633,广东,都是自用分享，别总蔽我，好吗👌,神秘,543,141,203480,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,NaN,NaN
3,65bde01900000000010310d3 65be15640000000011000...,🍬美丽心情永恒的阳光,52f38137b4c4d66b2eaeb2b6,https://www.xiaohongshu.com/user/profile/52f38...,https://www.xiaohongshu.com/explore/65bde01900...,祖玛珑香水推荐 周大福这款怎么不火？！ 今天武汉大雪🌨️ 过春节能干嘛之写字✍🏻 过春节在家...,2024-02-03 14:41:29,59,http://sns-webpic-qc.xhscdn.com/202403252147/f...,#祖玛珑##苦橙##香水推荐##英国梨与小苍兰# #周大福##周大福传承##周大福黄金# #...,...,951028702,湖北,ENTP丨斜杠青年丨体制内\n分享型人格丨喜欢一切美好事物\n…………\n“像一个新的大人，...,NaN,1115,128,31260,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,文化博主,NaN
4,63614f340000000007037280 6364c1de0000000008013...,满星欢喜,52ff7f7eb4c4d65a7e9329e4,https://www.xiaohongshu.com/user/profile/52ff7...,https://www.xiaohongshu.com/explore/63614f3400...,40块搞定玄关摆件 挂画太俗？玄关摆件第二弹～200元的快乐 为新家配置了三口锅，满足日常所...,2022-11-02 00:54:12,105,http://sns-webpic-qc.xhscdn.com/202403252207/b...,#玄关摆件##花器##玄关画##玄关##入户玄关##玄关设计##我的装修记录# #玄关过道走...,...,180263222,福建,装修小白一只\n家装完成啦 已入住一年多\n偶尔分享一些生活好物 \n随喜随性 祝大家生活愉快,NaN,146,167,189380,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,家居博主,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22034,65fe9d60000000000d00f160 65fedc52000000000d00f...,毕,65fd3a6d000000000600c11f,https://www.xiaohongshu.com/user/profile/65fd3...,https://www.xiaohongshu.com/explore/65fe9d6000...,劳动人就要戴劳力士，我也一劳永逸了！ 个人急出，年前朋友送的不太喜欢了！忍痛出 急出！！含泪...,2024-03-23 17:14:08,15,http://sns-webpic-qc.xhscdn.com/202403252325/9...,#劳力士##劳力士日志##劳力士rolex# #日志##手表# #女表##日志# #劳力士#...,...,42762125897,福建,还没有简介,NaN,0,1,126,,NaN,NaN
22035,65ffe1c00000000012021877 6600da76000000000d00dee7,七里香,65fd56d5000000000600c62b,https://www.xiaohongshu.com/user/profile/65fd5...,https://www.xiaohongshu.com/explore/65ffe1c000...,"看了男友手机 我刷他卡买了块冰莓粉 却.... 个人急售！！忍痛亏出,暂挂3天",2024-03-24 16:18:08,5,http://sns-webpic-qc.xhscdn.com/202403252327/4...,#劳力士##rolex# #日志##女表#,...,26315541830,福建,还没有简介,NaN,0,1,22,,NaN,NaN
22036,65fdb3ed000000001203ebaf 65fdbcfe000000000d00f...,煎饼果子,65fdb31b000000000600c4ac,https://www.xiaohongshu.com/user/profile/65fdb...,https://www.xiaohongshu.com/explore/65fdb3ed00...,现代艺术,2024-03-23 00:38:05,17,http://sns-webpic-qc.xhscdn.com/202403252135/f...,,...,42767838188,广东,还没有简介,NaN,3,1,78,,NaN,NaN
22037,66013bf60000000012023c8d 66013d80000000000d00c...,山橘,65fe598a000000000b030ea1,https://www.xiaohongshu.com/user/profile/65fe5...,https://www.xiaohongshu.com/explore/66013bf600...,记录一下人流过程 做完人流手术，分享一下过程 分手后，发现怀孕，就做了人流手术 做人流手术，...,2024-03-25 16:55:18,5,http://sns-webpic-qc.xhscdn.com/202403252358/d...,,...,11889112600,安徽,沉默是金,NaN,0,0,5,,NaN,NaN


In [7]:
#清洗特殊符号
# def clean(text):
#     # 确保text是字符串
#     text = str(text)
#     text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", text)  # 去除正文中的@和回复/转发中的用户名
#     text = re.sub(r"\[\S+\]", "", text)      # 去除表情符号
#     # text = re.sub(r"#\S+#", "", text)      # 保留话题内容
#     URL_REGEX = re.compile(
#         r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
#         re.IGNORECASE)
#     text = re.sub(URL_REGEX, "", text)       # 去除网址
#     text = text.replace("转发微博", "")       # 去除无意义的词语
#     text = re.sub(r"\s+", " ", text)        # 合并正文中过多的空格
#     return text.strip()

# data['作品标题'] = data['作品标题'].apply(clean)
# data['作品标题']
###########################################################################################################
# data_copy = data.copy()

# def remove_characters1(text):
#     # The updated patterns should avoid matching Chinese characters and normal text.
#     # Adjusting the regex patterns to make sure they do not match digits and letters unnecessarily
#     characters = [
#         '#E-s\d{2}',    # Specific emoticon pattern
#         '%\d+',         # Percentage followed by digits
#         '/:\d{3}',      # Specific pattern that includes colon and digits
#         '/:(\w*)',      # Colon followed by any word character
#         '/:-([A-Za-z]+)', # Specific pattern with colon, dash and letters
#         '[\U00010000-\U0010ffff]', # Matches characters in a specific Unicode range
#         '\*&\*'         # Matches the specific pattern *&*
#     ]
#     patterns = '|'.join(characters)
#     # Compile the pattern for better performance
#     pattern = re.compile(patterns)
#     # Substitute the matched patterns with empty string
#     processed_text = pattern.sub('', text)
#     return processed_text

# # Step 2: Update the function to avoid removing Chinese punctuation
# def remove_characters2(desstr):
#     # Define a list of punctuation to be removed, excluding Chinese punctuation
#     punctuation = '＂"\＃#*＄$％＆＇\'().^_^＊＋－/:；＜＝＞<>＠［＼］＾＿｀｛｜｝～｟｠｢｣､\u3000、〃〈〉《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏﹑﹔\\·?!'
#     # Remove the punctuation from the string
#     tr = re.sub('[{}]'.format(punctuation), "", desstr)
#     # Pattern to match text like [emoji]
#     pattern = r"\[[^\[\]]*\]"
#     # Substitute the matched patterns with empty string
#     tr = re.sub(pattern, "", tr)
#     return tr

# # Step 3: Keep remove_characters3 as is, since it targets specific Unicode characters for emojis
# def remove_characters3(desstr1):
#     # This pattern matches specific Unicode characters used as emojis
#     pattern = re.compile(r'[\u2022\ufecc\ufe8e\u2764]')
#     # Substitute the matched patterns with empty string
#     processed_text = re.sub(pattern, '', desstr1)
#     return processed_text

# # 遍历所有列并安全地应用清洗函数
# data_copy = data.copy()
# for column in data_copy.columns:
#     data_copy[column] = data_copy[column].apply(remove_characters1)
#     data_copy[column] = data_copy[column].apply(remove_characters2)
#     data_copy[column] = data_copy[column].apply(remove_characters3)
    
# data_copy

In [8]:
# # 保存清洗后的输出文件
# data.to_excel('../输出文件/已清洗文本.xlsx', index=False, engine='openpyxl')

In [9]:
# 对评论进行分词处理
## 加载停用词文件
def load_stopwords(paths):
    stopwords = set()
    for path in paths:
        with open(path, 'r', encoding='utf-8') as file:
            stopwords.update(line.strip() for line in file)
    return stopwords
stopwords_paths = ["../中文停用词/cn_stopwords.txt",
                       "../中文停用词/hit_stopwords.txt",
                       "../中文停用词/scu_stopwords.txt",
                       "../中文停用词/baidu_stopwords.txt"]
stopwords = load_stopwords(stopwords_paths)

In [10]:
## 分词
def tokenize_and_filter(text, stopwords):
    words = jieba.cut(text)
    return [word for word in words if word not in stopwords and word.strip()]
data_fenci = clean_data['作品正文内容'].apply(lambda x: tokenize_and_filter(x, stopwords))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.708 seconds.
Prefix dict has been built successfully.


In [11]:
comments = data_fenci
comments

0        [家里, 儿童房, 无奈, 宝宝, 独立, 空间, 图四是, 改造, 前, 只能, 利用, ...
1        [霍, 格沃兹, 同学会, 本蛇院, 女孩, 第一个, 📍, 话, 说, 先上, 定位, 📍...
2        [36, 宝妈, 挑战, 有娃, 少, 穿, 裙子, 运动裤, 运动裤, 一眼, 相, 中,...
3        [入, 苦, 橙, 小苍兰, 真的, 惊喜, R, 细说, ~, 苦, 橙, 香水, emm...
4        [这款, 花器, 听说, 店家, 做, 家里, 换, 新, 花器, 摆件, 喜欢, 朋友, ...
                               ...                        
22034    [妇女节, 喜提, 劳力士, 银盘, 31mm, 日志, 银色, 真的, 太太, 太, 温柔...
22035    [无意间, 手机, ...., 报复, 刷, 卡, 找, 店主, 小姐姐, 定制, 一只, ...
22036       [屏风, 背景墙, 自动, 转向灯, 灯, 智能, 旋转, 触摸, 调光, 壁灯, 吸顶灯]
22037    [希望, 分享, 人流, 过程, 面临, 情况, 姐妹, 参考, 术, 前, 挂号, 面诊,...
22038    [春天里, 遛, 遛, 话题, 风铃, 话题, 风铃, 花, 话题, 🌈, 最重, 135,...
Name: 作品正文内容, Length: 22032, dtype: object

In [12]:
# 引用参考人群聚类文档
excel_path = "../自定人群0410.xlsx"
def read_dimensions_from_excel(excel_path):
    df = pd.read_excel(excel_path)
    dimensions = {}
    for column in df.columns:
        dimensions[column] = df[column].dropna().tolist()
    return dimensions
# cluster_reference = pd.read_excel('../人群分词.xlsx')
dimensions = read_dimensions_from_excel(excel_path)
dimensions

{'晚睡娱乐的夜猫子': ['深夜游戏',
  '夜间剧集',
  '深夜食堂',
  '夜猫子',
  '深夜随笔',
  '夜晚社交',
  '连续剧',
  '游戏攻略',
  '深夜追剧',
  '通宵',
  '熬夜',
  '刷手机',
  '看直播',
  '深夜网易云',
  '网抑云',
  '玩游戏',
  '深夜',
  '晚上追剧',
  '晚上刷手机',
  '打游戏'],
 '熬夜失眠的职场打工人': ['加班熬夜',
  '职场',
  '压力',
  '失眠',
  '夜间思绪',
  '焦虑缓解',
  '压力管理',
  '工作与健康',
  '职场生存',
  '时间管理',
  '996',
  '失眠',
  '加班',
  '熬夜',
  '职场压力',
  '担心裁员',
  '焦虑',
  '工作',
  '未来迷茫',
  '工作邮件',
  '晚上加班',
  '工作梦',
  '咖啡',
  '个人发展',
  '打工人'],
 '哄睡喂奶宝妈宝爸': ['哄睡',
  '宝宝喂奶',
  '育儿',
  '婴儿护理',
  '宝妈',
  '宝爸',
  '家庭教育',
  '亲子时间',
  '共读绘本',
  '睡前故事',
  '绘本',
  '育儿',
  '母婴',
  '宝宝',
  '喂奶',
  '育儿',
  '啼哭',
  '婴儿',
  '母乳',
  '尿布',
  '奶爸'],
 '共读睡前绘本的青少年': ['睡前',
  '绘本',
  '晚上共读',
  '童书',
  '故事时间',
  '宝贝睡前故事',
  '亲子阅读',
  '绘本馆',
  '童话世界',
  '成长故事',
  '亲子',
  '阅读习惯',
  '绘本选择',
  '教育启蒙',
  '语言发展',
  '想象力培养',
  '家庭教育',
  '儿童文学',
  '晚安故事',
  '情感交流',
  '睡前故事'],
 '健康养生的自律达人': ['健康养生',
  '自律生活',
  '日常养生',
  '健康饮食',
  '健康',
  '养生',
  '冥想',
  '瑜伽',
  '晨练',
  '宠物',
  '自律',
  '猫',
  '猫狗'],
 '生活格调中高层管理': ['

In [13]:
# 加载腾讯AI实验室的词向量模型

word_vectors = KeyedVectors.load_word2vec_format('../Tencent Embedding/tencent-ailab-embedding-zh-d200-v0.2.0-s.txt', binary=False)


In [14]:
# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [15]:
# 创建一个新的字典，用于存储每个维度下特征词的向量张量
dimensions_vectors = {}

for dimension, keywords in dimensions.items():
    # 存储当前维度下的所有特征词向量张量
    vectors = []
    for keyword in keywords:
        # 检查词向量模型中是否有该词
        if keyword in word_vectors.key_to_index:
            # 将词向量转换为PyTorch张量，并加载到GPU
            vector_tensor = torch.tensor(word_vectors[keyword], dtype=torch.float32, device=device)
            vectors.append(vector_tensor)
    # 将转换后的向量张量列表存储在新字典中
    dimensions_vectors[dimension] = vectors

In [16]:
# 设计聚类的名称
def read_cluster_names(excel_path):
    df = pd.read_excel(excel_path)
    return df.columns.tolist()  # 假设聚类名在第一行
cluster_names = read_cluster_names(excel_path)

# 几种打分方法

In [17]:
# # 计算方法1 计算评论中与每个维度关键词匹配的词语比例。这种方法是基于关键词匹配进行的简单评分
# def score_comment(comment_words, dimensions):
#     scores = {}
#     # 检查评论词语列表是否为空
#     if len(comment_words) == 0:
#         # 如果为空，为每个维度赋予默认分数0（或其他值）
#         for dimension in dimensions.keys():
#             scores[dimension] = 0
#         return scores
    
#     for dimension, keywords in dimensions.items():
#         matched_words = [word for word in comment_words if word in keywords]
#         # 由于已经确保了评论词语列表不为空，这里不会发生除以零的情况
#         score = len(matched_words) / len(comment_words)
#         scores[dimension] = score
#     return scores

# # 对每个评论计算打分
# scores_list = comments.apply(lambda x: score_comment(x, dimensions))

# # 转换成DataFrame以便查看
# scores_df = pd.DataFrame(list(scores_list), index=comments.index)
# scores_df

In [18]:
# # 计算方法2 使用Jaccard相似度为评论打分
# def calculate_jaccard_similarity(set1, set2):
#     """
#     计算两个集合之间的Jaccard相似度。
#     """
#     intersection = len(set1.intersection(set2))
#     union = len(set1.union(set2))
#     if union == 0:
#         return 0
#     else:
#         return intersection / union

# def score_comment_with_jaccard(comment_words, dimensions):
#     """
#     使用Jaccard相似度为评论打分。
#     """
#     scores = {}
#     comment_words_set = set(comment_words)  # 将评论词列表转换为集合
#     if len(comment_words_set) == 0:
#         # 如果评论为空，为每个维度赋予默认分数0
#         for dimension in dimensions.keys():
#             scores[dimension] = 0
#     else:
#         for dimension, keywords in dimensions.items():
#             keywords_set = set(keywords)  # 将维度关键词列表转换为集合
#             # 计算Jaccard相似度
#             score = calculate_jaccard_similarity(comment_words_set, keywords_set)
#             scores[dimension] = score
#     return scores

# # 假设 comments 是一个包含分词后评论的pandas Series，dimensions 是维度关键词字典
# scores_list = comments.apply(lambda x: score_comment_with_jaccard(x, dimensions))

# # 转换成DataFrame以便查看
# scores_df = pd.DataFrame(list(scores_list), index=comments.index)

# scores_df


In [19]:
#计算方法3 使用腾讯词向量结合Jaccard

# def calculate_cosine_similarity(vector1, vector2):
#     # 注意防止除零错误
#     if np.any(vector1) and np.any(vector2):
#         return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]
#     else:
#         return 0

# def get_vector(word):
#     try:
#         return word_vectors[word]
#     except KeyError:
#         # 如果词不在词向量模型中，则返回None
#         return None

# def score_comment_with_jaccard(comment_words, dimensions):
#     scores = {}
#     for dimension, keywords in dimensions.items():
#         dimension_scores = []
#         for word in comment_words:
#             word_vector = get_vector(word)
#             if word_vector is not None:
#                 # 计算每个评论词与关键词之间的最大相似度
#                 max_similarity = max([calculate_cosine_similarity(word_vector, get_vector(keyword)) for keyword in keywords if get_vector(keyword) is not None], default=0)
#                 dimension_scores.append(max_similarity)
#         # 计算该维度下的平均相似度作为得分
#         if dimension_scores:
#             scores[dimension] = np.mean(dimension_scores)
#         else:
#             scores[dimension] = 0
#     return scores

# scores_list = comments.apply(lambda x: score_comment_with_jaccard(x, dimensions))

# # 转换成DataFrame以便查看
# scores_df = pd.DataFrame(list(scores_list), index=comments.index)

# scores_df
##########################################################################################
# 词向量缓存
word_tensor_cache = {}

def get_word_tensor(word, word_vectors, device):
    if word not in word_tensor_cache:
        word_tensor_cache[word] = torch.tensor(word_vectors[word], device=device)
    return word_tensor_cache[word]

# 修改 calculate_similarity_for_comment 以使用缓存
def calculate_similarity_for_comment(comment_words, dimensions_vectors, word_vectors, device='cuda'):
    # 使用缓存转换评论中的词为向量，并堆叠成矩阵
    comment_vectors = [get_word_tensor(word, word_vectors, device) for word in comment_words if word in word_vectors]
    if not comment_vectors:
        return {}
    comment_matrix = torch.stack(comment_vectors)
    
    scores = {}
    for dimension, feature_vectors in dimensions_vectors.items():
        # 计算余弦相似度
        sims = torch.stack([torch.max(F.cosine_similarity(comment_matrix, fv.unsqueeze(0).expand_as(comment_matrix), dim=1)) for fv in feature_vectors])
        average_similarity = sims.mean().item() if sims.numel() > 0 else 0
        scores[dimension] = average_similarity
    return scores



In [20]:
# 确保按照函数定义的参数顺序和名称传递参数
# 提取前300个评论
# comments_subset = comments.iloc[:3000]

similarity_scores_subset = comments.apply(lambda x: calculate_similarity_for_comment(x, dimensions_vectors, word_vectors, device='cuda'))

# 转化成DataFrame
similarity_scores_df = pd.DataFrame(similarity_scores_subset.tolist())



In [21]:
# # 筛选掉打分是0的帖子
# similarity_scores_df = similarity_scores_df.loc[~(similarity_scores_df==0).all(axis=1)]

# 计算每条评论的最大相似度得分
max_similarity_scores = similarity_scores_df.max(axis=1)

# 设置阈值
threshold = 0.55

# 基于阈值筛选评论
filtered_indexes = max_similarity_scores[max_similarity_scores > threshold].index
filtered_similarity_scores_df = similarity_scores_df.loc[filtered_indexes]
filtered_similarity_scores_df


,晚睡娱乐的夜猫子,熬夜失眠的职场打工人,哄睡喂奶宝妈宝爸,共读睡前绘本的青少年,健康养生的自律达人,生活格调中高层管理,专注家居家装的博主商户
0,0.401844,0.407992,0.593441,0.570328,0.454146,0.512157,0.638903
1,0.541838,0.493669,0.615401,0.561297,0.544661,0.556414,0.539482
2,0.492381,0.520847,0.603825,0.431534,0.499540,0.470051,0.448471
3,0.563102,0.563047,0.563937,0.531373,0.585584,0.562190,0.475426
4,0.609869,0.565662,0.574234,0.487382,0.579984,0.558008,0.620719
...,...,...,...,...,...,...,...
22022,0.522869,0.498701,0.550041,0.579603,0.571802,0.560504,0.644545
22023,0.456636,0.497150,0.444431,0.391244,0.482283,0.474139,0.593087
22026,0.601295,0.546088,0.515729,0.516468,0.583044,0.473451,0.384258
22030,0.526212,0.647664,0.604306,0.469851,0.579508,0.484415,0.567397


In [22]:
# 选择每个评论最高分的维度
scores_max = filtered_similarity_scores_df.max(axis=1)
max_scores = filtered_similarity_scores_df.idxmax(axis=1)
scores_max,max_scores

(0        0.638903
 1        0.615401
 2        0.603825
 3        0.585584
 4        0.620719
            ...   
 22022    0.644545
 22023    0.593087
 22026    0.601295
 22030    0.647664
 22031    0.596666
 Length: 18806, dtype: float64,
 0        专注家居家装的博主商户
 1           哄睡喂奶宝妈宝爸
 2           哄睡喂奶宝妈宝爸
 3          健康养生的自律达人
 4        专注家居家装的博主商户
             ...     
 22022    专注家居家装的博主商户
 22023    专注家居家装的博主商户
 22026       晚睡娱乐的夜猫子
 22030     熬夜失眠的职场打工人
 22031      健康养生的自律达人
 Length: 18806, dtype: object)

In [23]:
clean_data['cluster'] = max_scores
clean_data['scores'] = scores_max
clean_data

/tmp/ipykernel_61408/2774627725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['cluster'] = max_scores
/tmp/ipykernel_61408/2774627725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['scores'] = scores_max


,笔记id,作者昵称,作者id,作者链接,作品链接,作品标题,发布时间,图片数量,图片列表,关联话题名称,...,简介,location,关注,粉丝,获赞与收藏,info,profession,college,cluster,scores
0,65ab78e7000000000a030984 65ddc72f000000000b00edeb,鱼的宇宙,52732665b4c4d6556e9b39f1,https://www.xiaohongshu.com/user/profile/52732...,https://www.xiaohongshu.com/explore/65ab78e700...,2岁宝宝的阅读角 床头柜区域改造 边住边改之：电视柜改绘本架,2024-01-20 15:40:23,7,http://sns-webpic-qc.xhscdn.com/202403252132/c...,#儿童书架##家居改造##房间改造##亲子共读##读书角布置##读书角##绘本架##惬意时光...,...,工作之外，才是生活。,北京朝阳区,312,43,38,女 女,视觉设计师,NaN,专注家居家装的博主商户,0.638903
1,6022733c0000000021037a82 6587fdce0000000009018...,甜橙小布丁🍮,52e0fc98b4c4d626cd1fa18e,https://www.xiaohongshu.com/user/profile/52e0f...,https://www.xiaohongshu.com/explore/6022733c00...,环球影城｜哈利波特｜魔杖触发事件 就在武汉！人少！好看！免费！的国风美术馆 左滑查看成片！成...,2021-02-09 19:34:20,148,http://sns-webpic-qc.xhscdn.com/202403252239/c...,#武汉吃喝玩乐##武汉旅游攻略##武汉拍照##武汉打卡##周末去哪儿##宝藏店铺##假日c...,...,💕💕💕,NaN,396,1629,550980,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,摄影博主,NaN,哄睡喂奶宝妈宝爸,0.615401
2,658d1a28000000001d0303e4 6592716d000000001101d...,大耳朵猫酱,52e5fca0b4c4d61c32e520d2,https://www.xiaohongshu.com/user/profile/52e5f...,https://www.xiaohongshu.com/explore/658d1a2800...,"OAKMOO""忍冬灵药""～小个子天菜 新年第一天的快乐来自Moi Par暖 生椰拿铁get ...",2023-12-28 14:48:08,103,http://sns-webpic-qc.xhscdn.com/202403252335/5...,#王者毅也的笔记#,...,都是自用分享，别总蔽我，好吗👌,神秘,543,141,203480,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,NaN,NaN,哄睡喂奶宝妈宝爸,0.603825
3,65bde01900000000010310d3 65be15640000000011000...,🍬美丽心情永恒的阳光,52f38137b4c4d66b2eaeb2b6,https://www.xiaohongshu.com/user/profile/52f38...,https://www.xiaohongshu.com/explore/65bde01900...,祖玛珑香水推荐 周大福这款怎么不火？！ 今天武汉大雪🌨️ 过春节能干嘛之写字✍🏻 过春节在家...,2024-02-03 14:41:29,59,http://sns-webpic-qc.xhscdn.com/202403252147/f...,#祖玛珑##苦橙##香水推荐##英国梨与小苍兰# #周大福##周大福传承##周大福黄金# #...,...,ENTP丨斜杠青年丨体制内\n分享型人格丨喜欢一切美好事物\n…………\n“像一个新的大人，...,NaN,1115,128,31260,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,文化博主,NaN,健康养生的自律达人,0.585584
4,63614f340000000007037280 6364c1de0000000008013...,满星欢喜,52ff7f7eb4c4d65a7e9329e4,https://www.xiaohongshu.com/user/profile/52ff7...,https://www.xiaohongshu.com/explore/63614f3400...,40块搞定玄关摆件 挂画太俗？玄关摆件第二弹～200元的快乐 为新家配置了三口锅，满足日常所...,2022-11-02 00:54:12,105,http://sns-webpic-qc.xhscdn.com/202403252207/b...,#玄关摆件##花器##玄关画##玄关##入户玄关##玄关设计##我的装修记录# #玄关过道走...,...,装修小白一只\n家装完成啦 已入住一年多\n偶尔分享一些生活好物 \n随喜随性 祝大家生活愉快,NaN,146,167,189380,女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女 女,家居博主,NaN,专注家居家装的博主商户,0.620719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22034,65fe9d60000000000d00f160 65fedc52000000000d00f...,毕,65fd3a6d000000000600c11f,https://www.xiaohongshu.com/user/profile/65fd3...,https://www.xiaohongshu.com/explore/65fe9d6000...,劳动人就要戴劳力士，我也一劳永逸了！ 个人急出，年前朋友送的不太喜欢了！忍痛出 急出！！含泪...,2024-03-23 17:14:08,15,http://sns-webpic-qc.xhscdn.com/202403252325/9...,#劳力士##劳力士日志##劳力士rolex# #日志##手表# #女表##日志# #劳力士#...,...,还没有简介,NaN,0,1,126,,NaN,NaN,NaN,NaN
22035,65ffe1c00000000012021877 6600da76000000000d00dee7,七里香,65fd56d5000000000600c62b,https://www.xiaohongshu.com/user/profile/65fd5...,https://www.xiaohongshu.com/explore/65ffe1c000...,"看了男友手机 我刷他卡买了块冰莓粉 却.... 个人急售！！忍痛亏出,暂挂3天",2024-03-24 16:18:08,5,http://sns-webpic-qc.xhscdn.com/202403252327/4...,#劳力士##rolex# #日志##女表#,...,还没有简介,NaN,0,1,22,,NaN,NaN,NaN,NaN
22036,65fdb3ed000000001203ebaf 65fdbcfe000000000d00f...,煎饼果子,65fdb31b000000000600c4ac,https://www.xiaohongshu.com/user/profile/65fdb...,https://www.xiaohongshu.com/explore/65fdb3ed00...,现代艺术,2024-03-23 00:38:05,17,http://sns-webpic-qc.xhscdn.com/202403252135/f...,,...,还没有简介,NaN,3,1,78,,NaN,NaN,NaN,NaN
22037,66013bf60000000012023c8d 66013d80000000000d00c...,山橘,65fe598a000000000b030ea1,https://www.xiaohongshu.com/user/profile/65fe5...,https://www.xiaohongshu.com/explore/66013bf600...,记录一下人流过程 做完人流手术，分享一下过程 分手后，发现怀孕，就做了人流手术 做人流手术，...,2024-03-25 16:55:18,5,http://sns-webpic-qc.xhscdn.com/202403252358/d...,,...,沉默是金,NaN,0,0,5,,NaN,NaN,NaN,NaN


In [24]:
# 计算每个类别的数量
category_counts = max_scores.value_counts()

# 计算总数
total_count = max_scores.size

# 计算每个类别的占比
category_proportions = category_counts / total_count

category_counts,category_proportions

(专注家居家装的博主商户    9438
 健康养生的自律达人      3157
 哄睡喂奶宝妈宝爸       2630
 晚睡娱乐的夜猫子       1557
 熬夜失眠的职场打工人     1519
 生活格调中高层管理       348
 共读睡前绘本的青少年      157
 Name: count, dtype: int64,
 专注家居家装的博主商户    0.501861
 健康养生的自律达人      0.167872
 哄睡喂奶宝妈宝爸       0.139849
 晚睡娱乐的夜猫子       0.082793
 熬夜失眠的职场打工人     0.080772
 生活格调中高层管理      0.018505
 共读睡前绘本的青少年     0.008348
 Name: count, dtype: float64)

In [25]:
# 导出不同人群的帖子

# 确定保存文件的基础路径
base_path = '../output folder'

# 检查并创建基础路径
if not os.path.exists(base_path):
    os.makedirs(base_path)

# 按聚类人群名称分组保存到不同的Excel文件
for cluster_name, group_df in clean_data.groupby('cluster'):
    # 构建文件名
    filename = '0410-2/'+f"D2-1_小红书_{cluster_name}.xlsx"
    file_path = os.path.join(base_path, filename)
    
    # 保存到Excel文件
    group_df.to_excel(file_path, index=False)
    
    print(f"Saved {cluster_name} posts to {file_path}")

Saved 专注家居家装的博主商户 posts to ../output folder/0410-2/D2-1_小红书_专注家居家装的博主商户.xlsx
Saved 健康养生的自律达人 posts to ../output folder/0410-2/D2-1_小红书_健康养生的自律达人.xlsx
Saved 共读睡前绘本的青少年 posts to ../output folder/0410-2/D2-1_小红书_共读睡前绘本的青少年.xlsx
Saved 哄睡喂奶宝妈宝爸 posts to ../output folder/0410-2/D2-1_小红书_哄睡喂奶宝妈宝爸.xlsx
Saved 晚睡娱乐的夜猫子 posts to ../output folder/0410-2/D2-1_小红书_晚睡娱乐的夜猫子.xlsx
Saved 熬夜失眠的职场打工人 posts to ../output folder/0410-2/D2-1_小红书_熬夜失眠的职场打工人.xlsx
Saved 生活格调中高层管理 posts to ../output folder/0410-2/D2-1_小红书_生活格调中高层管理.xlsx
